<a href="https://colab.research.google.com/github/woo2201/ColabDrive/blob/reservior/JobMarket_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/dataset/JobMarket/

/content/gdrive/MyDrive/dataset/JobMarket


In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import requests
import lxml
import re
import html
import urllib

In [4]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 5.6 MB/s 
     |████████████████████████████████| 448 kB 57.2 MB/s 


In [5]:
from konlpy.tag import Okt
okt = Okt()

링크 : http://comp.fnguide.com/SVO2/ASP/SVD_UJRank.asp

해당 링크에는 500대 한국 기업의 전반적인 재무재표성 데이터가 있습니다.
상위 500개 기업을 나열해서, 매출액 및 자산규모/매출액과 채용 규모 및 만족도에 어떠한 관계가 있는지 살펴보려 합니다.

1. 우리의 고용자가 될 수 있는 500대 기업의 정보에 대해 긁어옵니다.
물론 더욱 다양한 기업들이 있습니다만, 많은 구직자들이 

In [6]:
companies = pd.read_csv('companyInfo.csv')

In [7]:
companies.loc[2, '종목명'] = '한국전력공사'

In [33]:
class infoParser:
    def __init__(self, company):
        self.company = company
        try:
            self.html = urlopen("https://www.teamblind.com/kr/company/" + urllib.parse.quote(str(self.company)))  
            self.bsObject = BeautifulSoup(self.html, "html.parser", from_encoding='utf-8')
        except urllib.error.HTTPError as err:
            pass     

    def description(self):
        try:
            val = self.bsObject.find('div', {"class":"desc all"}).text
        except AttributeError as e:
            return None
        return val

    def overallRating(self):
        try:
            val = self.bsObject.find('span', {"class" : "star"}).text
        except AttributeError as e:
            return None
        return val

    def individualStar(self):
        try:
            val = [float(i.text) for i in self.bsObject.find_all('strong',{'class' : "star"})]
        except AttributeError as e:
            return None
        return val
    
    def reviewCount(self):
        try:
            val = self.bsObject.find('em',{'class' : "count"}).text
        except AttributeError as e:
            return None
        return val

    def industryCategory(self):
        try:
            val = self.bsObject.select_one('#wrap > section > div > div > div.cpctw.cpovw > div > div > div > section.overview > ul > li:nth-of-type(2)').text
        except AttributeError as e:
            return None
        return val 

    def companyLocation(self):
        try:
            val = self.bsObject.select_one('#wrap > section > div > div > div.cpctw.cpovw > div > div > div > section.overview > ul > li:nth-of-type(3)').text
        except AttributeError as e:
            return None
        return val 
        
    def reviewText(self):
        try:
            self.html = urlopen("https://www.teamblind.com/kr/company/" + urllib.parse.quote(str(self.company)) + '/reviews')  
            self.bsObject = BeautifulSoup(self.html, "html.parser", from_encoding='utf-8')
            val = self.bsObject.select_one('#wrap > section > div > div > div.cpctw > div > div > div > div > section:nth-of-type(2) > div.review_all > div:nth-of-type(1) > div > div.review_item_inr').text.split('  장점')[-1]
        except (AttributeError, HTTPError) as e:
            return None
        return val
    



In [34]:
df = pd.DataFrame({'company' : [],
                   'description': [],
                   'overallRating': [],
                   'individualStar' : [],
                   'reviewCount': [],
                   'industryCat': [],
                   'companyLocation': [],
                   'reviewText' : []})# starName = [str(i.text) for i in bsObject.find_all('span', {'class' : 'desc'})][0:5]; starName

for cpny in companies['종목명']:
    test = infoParser(company = cpny)
    adix = {'company' : test.company,
            'description': test.description(),
            'overallRating': test.overallRating(),
            'individualStar' : test.individualStar(),
            'reviewCount': test.reviewCount(),
            'industryCat': test.industryCategory(),
            'companyLocation' : test.companyLocation(),
            'reviewText' : test.reviewText()}
    
    df = df.append(adix, ignore_index = True)


In [35]:
df.sample(4)

,company,description,overallRating,individualStar,reviewCount,industryCat,companyLocation,reviewText
24,LG생활건강,"LG생활건강은 생활용품, 화장품, 음료 제조 및 판매업을 주로 영위하는 기업으로 치...",Rating Score2.6,"[2.5, 3.4, 2.2, 2.6, 1.9]",538개 리뷰,업계\n 화장품 제조/판매업\n,본사\n 서울특별시 종로구\n,그나마 휴가는 자유롭게 쓸 수 있음30자 쓰라는데 더 장점을 못찾겠음 단점성과에 대...
407,비에이치,비에이치는 전자제품의 부품인 FPCB를 생산하는 회사입니다. 비에이치는 안정된 고객...,Rating Score2.1,"[2.4, 2.7, 1.9, 2.0, 1.7]",23개 리뷰,업계\n 반도체/전자기기 제조업 \n,본사\n 인천광역시 부평구\n,관리직들은 좀자유롭게 근무시간에 휴식을 취할수있다.복지(수당)가 그나마 생산직이 좋...
256,보령제약,"보령제약은 완제 의약품 제조업체로서 각종 의약품, 소염진통제, 항생제 등을 제조·유...",Rating Score3.6,"[3.3, 3.6, 3.7, 3.2, 3.0]",272개 리뷰,업계\n 의약품 제조/판매업\n,본사\n 서울특별시 종로구\n,장점? 인센 꽤 나옴. 월급 꽤 괜찮음. 그리고 없다 단점예전엔 그렇게 나쁘지 않...
162,티씨케이,"티씨케이는 가공 공정, 고순화 공정, 코팅 공정 과정을 거쳐 반도체, LED, 태양...",Rating Score3.1,"[2.9, 3.3, 3.8, 2.6, 2.3]",31개 리뷰,업계\n 기타 제조/수리업\n,본사\n 경기도 안성시\n,밥 맛있고 성과급 나오고 업무능력 성장시킬 수 있는 장점 단점근무지가 안성시골. 출...


In [36]:
df = df[~df.description.isnull()]

In [37]:
df['overallRating'] = df.overallRating.apply(lambda x : re.sub('[a-zA-z]','',x))

In [38]:
df['careerImprovement'] = df.individualStar.apply(lambda x: x[0] if x else '')
df['workLifeBalance'] = df.individualStar.apply(lambda x: x[1] if x else '')
df['salaryWelfare'] = df.individualStar.apply(lambda x: x[2] if x else '')
df['workplaceCulture'] = df.individualStar.apply(lambda x: x[3] if x else '')
df['boardMembers'] = df.individualStar.apply(lambda x: x[4] if x else '')

In [39]:
df

,company,description,overallRating,individualStar,reviewCount,industryCat,companyLocation,reviewText,careerImprovement,workLifeBalance,salaryWelfare,workplaceCulture,boardMembers
0,삼성전자,"삼성전자는 삼성그룹 계열사로 휴대폰, 컴퓨터, 네트워크시스템, 반도체부품, 디스플레...",3.5,"[3.2, 3.1, 3.7, 3.2, 2.6]","7,122개 리뷰",업계\n 반도체/전자기기 제조업 \n,본사\n 경기도 수원시\n,성과급이 잘 나오고 가깝다.밥이 맛있다. 반도체 업계 최고다 단점업무 강도가 세다....,3.2,3.1,3.7,3.2,2.6
1,SK하이닉스,"SK하이닉스는 반도체 전문기업으로 반도체, 컴퓨터, 통신기기 제조 등의 사업을 영위...",2.3,"[2.2, 2.3, 2.5, 2.2, 1.8]","3,282개 리뷰",업계\n 반도체/전자기기 제조업 \n,본사\n 경기도 이천시\n,제조기술이 아니면 좋은회사 급여수준은 높은편에 속함.. 단점24시간 반도체팹 가동인...,2.2,2.3,2.5,2.2,1.8
2,한국전력공사,한국전력공사는 국민경제 발전 기여를 목적으로 설립된 법인으로 시장형 공기업에 해당합...,4.0,"[3.4, 4.2, 3.9, 3.1, 3.0]","1,671개 리뷰",업계\n 국가/공공기관\n,본사\n 전라남도 나주시\n,"사기업 대비 업무 강도에 비해 괜찮은 연봉(하지만 상승률은,,) 단점대민 업무가 많...",3.4,4.2,3.9,3.1,3
3,SK,"SK는 에너지, 화학, 정보통신, 반도체, 마케팅, 바이오, 물류 부문에서 사업을 ...",3.5,"[3.1, 4.5, 3.2, 3.7, 2.9]",17개 리뷰,업계\n 금융업\n,본사\n 서울 종로구\n,최고의 워라벨눈치 안보고 근태 자유로움개발욕심 있으면 원없이 가능 단점사람은 많은데...,3.1,4.5,3.2,3.7,2.9
4,SK이노베이션,"SK이노베이션은 에너지, 화학 분야를 선도하는 기업으로 석유개발, 정유, 화학, 윤...",3.9,"[3.6, 3.9, 3.8, 3.9, 3.3]",733개 리뷰,업계\n 석유정제품 제조업\n,본사\n 서울특별시 종로구\n,과거 최고수준의 급여과거 최고수준의 이미지와 위상의료비 지원과 비교적 울산 도심과 ...,3.6,3.9,3.8,3.9,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,LG디스플레이,"LG디스플레이는 OLED, Curved OLED 등의 제조 기술력을 보유하고 있는 ...",2.9,"[2.9, 2.9, 2.8, 2.7, 2.3]","1,671개 리뷰",업계\n 반도체/전자기기 제조업 \n,본사\n 서울특별시 영등포구\n,어중간한 연봉으로 어디든 못가게 만들고얼라인드 플레이를 만든다고 하지만 어중간한 용...,2.9,2.9,2.8,2.7,2.3
476,S-Oil,S-OIL은 수익성 있는 지속성장을 미션으로 하는 정유/석유화학 업계의 대표회사입니...,3.4,"[2.7, 3.7, 3.8, 2.7, 2.1]",212개 리뷰,업계\n 석유정제품 제조업\n,본사\n 서울특별시 마포구\n,정년이 보장된다집중휴가가 보장된다울산과 부산의 중간지점에 있다(해운대 출퇴근이 가능...,2.7,3.7,3.8,2.7,2.1
477,롯데쇼핑,"롯데쇼핑은 롯데그룹의 계열사로 백화점, 할인점, 전자제품전문점, 슈퍼, TV홈쇼핑 ...",2.6,"[2.4, 2.9, 2.4, 2.2, 1.8]",809개 리뷰,업계\n 종합 소매업\n,본사\n 서울특별시 중구\n,월급이 잘나오는거 말고는 장점 없습니다 백화점 10% 할인? 단점수직적인 조직문화 ...,2.4,2.9,2.4,2.2,1.8
478,삼성중공업,삼성중공업은 강선 건조업을 주 업으로 하는 삼성그룹의 계열사입니다. 다양한 종류의 ...,2.1,"[2.2, 3.3, 1.8, 2.1, 1.4]",750개 리뷰,업계\n 조선업\n,본사\n 경상남도 거제시\n,유연근무나 삼성그룹을 따라가는 복지 제도는 나쁘지 않은 수준 단점꼰대문화 사무직 차...,2.2,3.3,1.8,2.1,1.4


In [40]:
df['reviewCount'] = df['reviewCount'].apply(lambda x: re.sub('[a-zA-z가-힣]','',x) if x else '')
df['reviewCount'] = df['reviewCount'].apply(lambda x: x.replace(',','')).apply(lambda x: int(x) if x else 0)

In [41]:
df['repNouns'] = df.description.apply(lambda x: okt.nouns(x))

In [42]:
df['industry'] = df.industryCat.apply(lambda x: x.replace('업계','').replace('\n', ''))

In [45]:
df['companyLocation'] = df.companyLocation.apply(lambda x: x.replace('본사','').replace('\n',''))

In [47]:
df = df.drop(['description','individualStar','industryCat'], axis = 1)

In [48]:
df

,company,overallRating,reviewCount,companyLocation,reviewText,careerImprovement,workLifeBalance,salaryWelfare,workplaceCulture,boardMembers,repNouns,industry
0,삼성전자,3.5,7122,경기도 수원시,성과급이 잘 나오고 가깝다.밥이 맛있다. 반도체 업계 최고다 단점업무 강도가 세다....,3.2,3.1,3.7,3.2,2.6,"[전자, 삼성, 그룹, 계, 열사, 휴대폰, 컴퓨터, 네트워크, 시스템, 반도체, ...",반도체/전자기기 제조업
1,SK하이닉스,2.3,3282,경기도 이천시,제조기술이 아니면 좋은회사 급여수준은 높은편에 속함.. 단점24시간 반도체팹 가동인...,2.2,2.3,2.5,2.2,1.8,"[하이닉스, 반도체, 기업, 반도체, 컴퓨터, 통신, 기기, 제조, 등, 사업, 영...",반도체/전자기기 제조업
2,한국전력공사,4.0,1671,전라남도 나주시,"사기업 대비 업무 강도에 비해 괜찮은 연봉(하지만 상승률은,,) 단점대민 업무가 많...",3.4,4.2,3.9,3.1,3,"[한국전력, 공사, 국민, 경제, 발전, 기여, 목적, 설립, 법인, 시장, 공기업...",국가/공공기관
3,SK,3.5,17,서울 종로구,최고의 워라벨눈치 안보고 근태 자유로움개발욕심 있으면 원없이 가능 단점사람은 많은데...,3.1,4.5,3.2,3.7,2.9,"[에너지, 화학, 정보, 통신, 반도체, 마케팅, 바이오, 물류, 부문, 사업, 영...",금융업
4,SK이노베이션,3.9,733,서울특별시 종로구,과거 최고수준의 급여과거 최고수준의 이미지와 위상의료비 지원과 비교적 울산 도심과 ...,3.6,3.9,3.8,3.9,3.3,"[이노베이션, 에너지, 화학, 분야, 선도, 기업, 석유, 개발, 정유, 화학, 윤...",석유정제품 제조업
...,...,...,...,...,...,...,...,...,...,...,...,...
475,LG디스플레이,2.9,1671,서울특별시 영등포구,어중간한 연봉으로 어디든 못가게 만들고얼라인드 플레이를 만든다고 하지만 어중간한 용...,2.9,2.9,2.8,2.7,2.3,"[디스플레이, 등, 제조, 기술, 보유, 디스플레이, 전문, 회사, 디스플레이, 기...",반도체/전자기기 제조업
476,S-Oil,3.4,212,서울특별시 마포구,정년이 보장된다집중휴가가 보장된다울산과 부산의 중간지점에 있다(해운대 출퇴근이 가능...,2.7,3.7,3.8,2.7,2.1,"[은, 수익, 성장, 미션, 정유, 석유화학, 업계, 대표, 회사, 대한민국, 아시...",석유정제품 제조업
477,롯데쇼핑,2.6,809,서울특별시 중구,월급이 잘나오는거 말고는 장점 없습니다 백화점 10% 할인? 단점수직적인 조직문화 ...,2.4,2.9,2.4,2.2,1.8,"[롯데, 쇼핑, 롯데, 그룹, 계, 열사, 백화점, 할인점, 전자제품, 전문점, 슈...",종합 소매업
478,삼성중공업,2.1,750,경상남도 거제시,유연근무나 삼성그룹을 따라가는 복지 제도는 나쁘지 않은 수준 단점꼰대문화 사무직 차...,2.2,3.3,1.8,2.1,1.4,"[중공업, 강선, 건조, 업, 주, 업, 그룹, 계, 열사, 종류, 선박, 업계, ...",조선업


In [ ]:
secondary = df[['company','overallRating','repNouns']].explode('repNouns')

In [49]:
df.to_csv('/content/gdrive/MyDrive/dataset/JobMarket/completeList2.csv')

In [ ]:
secondary.to_csv('/content/gdrive/MyDrive/dataset/JobMarket/completeList_second.csv')